In [1]:
from glob import glob
import json
import random
import os

In [2]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions'
]

In [3]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

In [4]:
files = glob('/home/husein/ssd3/gov.my/ultrachat-*.jsonl')
files = [f for f in files if 'coder' not in f]
lines = 0
for f in files:
    with open(f) as fopen:
        for l in fopen:
            lines += 1
lines

236338

In [5]:
files

['/home/husein/ssd3/gov.my/ultrachat-jurnaldbp.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-jurnaldbp-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-maktabahalbakri.com.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-gov.my.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-crossref-melayu.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-lom-agc.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-astroawani-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-textbooks.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-crossref-melayu-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-epenerbitan-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-hansard-malay.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-ms-wikipedia.jsonl',
 '/home/husein/ssd3/gov.my/ultrachat-muftiwp.gov.my.texts.jsonl']

In [6]:
from tqdm import tqdm

data_instructions = []

count, count1 = 0, 0
for file in files:
    with open(file) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
#             if l[1]['content'] is None:
#                 print(l)
#                 continue
            
            if 'glaive_coder_raw_text' in file:
                code_instruct = True
            else:
                code_instruct = False
                
            context = l[0]['content']
            
            if not code_instruct:
                l = l[1:]
                
            inputs = []
            for no, r in enumerate(l):
                
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if not code_instruct and r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if not code_instruct and r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                if no == 0 and not code_instruct and ('crossref-melayu' in file or random.random() > 0.7):
                    s = f"{role}: {context}\n\n{current_text}"
                    count += 1
                else:
                    s = f"{role}: {current_text}"
                    
                inputs.append(s)
                count1 += 1

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue

            data = '\n'.join(inputs).strip()
            
            if not len(data):
                continue
                
            a = {
                'prompt_input': None,
                'input': data,
                'output': None
            }
            data_instructions.append(a)
        
        
len(data_instructions)

1734it [00:01, 1272.98it/s]
6440it [00:02, 2299.39it/s]
3350it [00:01, 1959.77it/s]
10128it [00:04, 2308.39it/s]
1296it [00:00, 1386.11it/s]
8044it [00:03, 2036.75it/s]
60198it [00:13, 4362.73it/s]
49842it [00:29, 1665.89it/s]
9959it [00:04, 2165.16it/s]
4567it [00:02, 2265.44it/s]
72538it [00:22, 3192.08it/s]
4408it [00:01, 2608.90it/s]
3834it [00:01, 2014.36it/s]


235373

In [7]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [8]:
from tqdm import tqdm

for i in tqdm(range(len(data_instructions))):
    generate_and_tokenize_prompt(data_instructions[i])

100%|███████████████████████████████████████████████████████████████████████████████████████| 235373/235373 [00:02<00:00, 100198.46it/s]


In [9]:
with open('prepared-malaysian-ultrachat-nocode.jsonl', 'w') as fopen:
    for l in data_instructions:
        fopen.write(f'{json.dumps(l)}\n')